In [1]:
import xarray as xr
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def calcAnom(ds,anom_name):
    # Save the original dates 
    time=ds.time
    
    # Get the time dimensions; ntimes=# of samples; nyrs=# of calendar years; nlead=# of foreecast leads
    ntimes=len(time)
    index=ds.time['time.year']
    yStrt=index[0]
    yLast=index[-1]
    nyrs=int(yLast-yStrt)
    nlead=int(len(index)/nyrs)
    
    # Compute climatology for each forecast lead 
    climo=[]
    for i in range(nlead):
        #AMJ 12/22/22 fixed indexing bug
        #da=ds[i:ntimes-1:nlead,:,:].mean(dim='time')
        da=ds[i:ntimes:nlead,:,:].mean(dim='time')
        climo.append(da.to_dataset(name='clim'))
    ds_clim=xr.combine_nested(climo,concat_dim='time')
    
    # Change time coordinate from calendar dates to integers [0:ntimes]
    days=[]
    for i in range(nyrs):
        days.append(np.arange(nlead))
    days=np.reshape(days,nyrs*nlead)
    
    ds['time']=days
    
    # Compute anomalies 
    anoms=ds.groupby(ds.time)-ds_clim
    
    # Rename anomaly 
    anom=anoms['clim'].rename(anom_name)
    
    # Remap time coordinate to original calendar dates
    anom['time']=time
    
    return anom

I'm working with 2 years of data for now: 2014-2015

Future edit: I need to rename the SST files so that they all have the same format (when I first downloaded, the sst were lowercase. this will be fixed eventually).

In [3]:
path = '/scratch/ldoyle4/ufs8/daily/mean/sst/'

years_14 = ['2014']
years_15 = ['2015']
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
days_01 = ['01']
#days_15 = ['15']

files_01_1 = [f'{path}SST_{year}{month}01.nc' for year in years_14 for month in months]
files_01_2 = [f'{path}sst_{year}{month}01.nc' for year in years_15 for month in months]

#files_15_1 = [f'{path}SST_{year}{month}{day}.nc' for year in years_14 for month in months for day in days_15]
#files_15_2 = [f'{path}SST_{year}{month}{day}.nc' for year in years_15 for month in months for day in days_15]

In [4]:
#sst_ufs_1='/scratch/ldoyle4/ufs8/daily/mean/sst/*01.nc'
#sst_ufs_15='/scratch/ldoyle4/ufs8/daily/mean/sst/*15.nc'

taux_ufs_1='/scratch/ldoyle4/ufs8/daily/mean/taux/*01.nc'
#taux_ufs_15='/scratch/ldoyle4/ufs8/daily/mean/taux/*15.nc'

tauy_ufs_1='/scratch/ldoyle4/ufs8/daily/mean/tauy/*01.nc'
#tauy_ufs_15='/scratch/ldoyle4/ufs8/daily/mean/tauy/*15.nc'

ssv_ufs_1='/scratch/ldoyle4/ufs8/daily/mean/ssv/*01.nc'
#ssv_ufs_15='/scratch/ldoyle4/ufs8/daily/mean/ssv/*15.nc'

ssu_ufs_1='/scratch/ldoyle4/ufs8/daily/mean/ssu/*01.nc'
#ssu_ufs_15='/scratch/ldoyle4/ufs8/daily/mean/ssu/*15.nc'

In [6]:
ds_sst_ufs_1 = xr.open_mfdataset(files_01_1+files_01_2, combine='nested',concat_dim='time',parallel=True)
#ds_sst_ufs_15 = xr.open_mfdataset(files_15_1+files_15_2, combine='nested',concat_dim='time',parallel=True)

In [7]:
ds_sst_ufs_1

<xarray.Dataset>
Dimensions:  (time: 840, lat: 721, lon: 1441)
Coordinates:
  * time     (time) object 2014-01-01 00:00:00 ... 2016-01-04 00:00:00
  * lat      (lat) float32 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
  * lon      (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.2 359.5 359.8 360.0
Data variables:
    SST      (time, lat, lon) float32 dask.array<chunksize=(35, 721, 1441), meta=np.ndarray>

In [8]:
#ds_sst_ufs_1=xr.open_mfdataset(sst_ufs_1,combine='nested',concat_dim='time',parallel=True)
#ds_sst_ufs_15=xr.open_mfdataset(sst_ufs_15,combine='nested',concat_dim='time',parallel=True)

ds_taux_ufs_1=xr.open_mfdataset(taux_ufs_1,combine='nested',concat_dim='time',parallel=True)
#ds_taux_ufs_15=xr.open_mfdataset(taux_ufs_15,combine='nested',concat_dim='time',parallel=True)

ds_tauy_ufs_1=xr.open_mfdataset(tauy_ufs_1,combine='nested',concat_dim='time',parallel=True)
#ds_tauy_ufs_15=xr.open_mfdataset(tauy_ufs_15,combine='nested',concat_dim='time',parallel=True)

ds_ssv_ufs_1=xr.open_mfdataset(ssv_ufs_1,combine='nested',concat_dim='time',parallel=True)
#ds_ssv_ufs_15=xr.open_mfdataset(ssv_ufs_15,combine='nested',concat_dim='time',parallel=True)

ds_ssu_ufs_1=xr.open_mfdataset(ssu_ufs_1,combine='nested',concat_dim='time',parallel=True)
#ds_ssu_ufs_15=xr.open_mfdataset(ssu_ufs_15,combine='nested',concat_dim='time',parallel=True)

In [9]:
#print(ds_sst_ufs_1)
#print(ds_sst_ufs_15)

In [10]:
sst_anom_ufs_1=calcAnom(ds_sst_ufs_1['SST'],'SST_anom')
taux_anom_ufs_1=calcAnom(ds_taux_ufs_1['taux'],'taux_anom')
tauy_anom_ufs_1=calcAnom(ds_tauy_ufs_1['tauy'],'tauy_anom')
ssu_anom_ufs_1=calcAnom(ds_ssu_ufs_1['SSU'],'SSU_anom')
ssv_anom_ufs_1=calcAnom(ds_ssv_ufs_1['SSV'],'SSV_anom')


#sst_anom_ufs_15=calcAnom(ds_sst_ufs_15['SST'],'SST_anom')

Once the anomalies are calculated, need to split them up by month. The only way I could figure it out that wouldn't take too much time was just indexing it for every 35 days. I'm sure there's an easier solution, but I didn't have time to write one out.

In [11]:
sst_2014_01 = sst_anom_ufs_1[0:35,:,:]
sst_2014_02 = sst_anom_ufs_1[35:70,:,:]
sst_2014_03 = sst_anom_ufs_1[70:105,:,:]
sst_2014_04 = sst_anom_ufs_1[105:140,:,:]
sst_2014_05 = sst_anom_ufs_1[140:175,:,:]
sst_2014_06 = sst_anom_ufs_1[175:210,:,:]
sst_2014_07 = sst_anom_ufs_1[210:245,:,:]
sst_2014_08 = sst_anom_ufs_1[245:280,:,:]
sst_2014_09 = sst_anom_ufs_1[280:315,:,:]
sst_2014_10 = sst_anom_ufs_1[315:350,:,:]
sst_2014_11 = sst_anom_ufs_1[350:385,:,:]
sst_2014_12 = sst_anom_ufs_1[385:420,:,:]
sst_2015_01 = sst_anom_ufs_1[420:455,:,:]
sst_2015_02 = sst_anom_ufs_1[455:490,:,:]
sst_2015_03 = sst_anom_ufs_1[490:525,:,:]
sst_2015_04 = sst_anom_ufs_1[525:560,:,:]
sst_2015_05 = sst_anom_ufs_1[560:595,:,:]
sst_2015_06 = sst_anom_ufs_1[595:630,:,:]
sst_2015_07 = sst_anom_ufs_1[630:665,:,:]
sst_2015_08 = sst_anom_ufs_1[665:700,:,:]
sst_2015_09 = sst_anom_ufs_1[700:735,:,:]
sst_2015_10 = sst_anom_ufs_1[735:770,:,:]
sst_2015_11 = sst_anom_ufs_1[770:805,:,:]
sst_2015_12 = sst_anom_ufs_1[805:840,:,:]

In [38]:
ssu_2014_01 = ssu_anom_ufs_1[0:35,:,:]
ssu_2014_02 = ssu_anom_ufs_1[35:70,:,:]
ssu_2014_03 = ssu_anom_ufs_1[70:105,:,:]
ssu_2014_04 = ssu_anom_ufs_1[105:140,:,:]
ssu_2014_05 = ssu_anom_ufs_1[140:175,:,:]
ssu_2014_06 = ssu_anom_ufs_1[175:210,:,:]
ssu_2014_07 = ssu_anom_ufs_1[210:245,:,:]
ssu_2014_08 = ssu_anom_ufs_1[245:280,:,:]
ssu_2014_09 = ssu_anom_ufs_1[280:315,:,:]
ssu_2014_10 = ssu_anom_ufs_1[315:350,:,:]
ssu_2014_11 = ssu_anom_ufs_1[350:385,:,:]
ssu_2014_12 = ssu_anom_ufs_1[385:420,:,:]
ssu_2015_01 = ssu_anom_ufs_1[420:455,:,:]
ssu_2015_02 = ssu_anom_ufs_1[455:490,:,:]
ssu_2015_03 = ssu_anom_ufs_1[490:525,:,:]
ssu_2015_04 = ssu_anom_ufs_1[525:560,:,:]
ssu_2015_05 = ssu_anom_ufs_1[560:595,:,:]
ssu_2015_06 = ssu_anom_ufs_1[595:630,:,:]
ssu_2015_07 = ssu_anom_ufs_1[630:665,:,:]
ssu_2015_08 = ssu_anom_ufs_1[665:700,:,:]
ssu_2015_09 = ssu_anom_ufs_1[700:735,:,:]
ssu_2015_10 = ssu_anom_ufs_1[735:770,:,:]
ssu_2015_11 = ssu_anom_ufs_1[770:805,:,:]
ssu_2015_12 = ssu_anom_ufs_1[805:840,:,:]

In [39]:
ssv_2014_01 = ssv_anom_ufs_1[0:35,:,:]
ssv_2014_02 = ssv_anom_ufs_1[35:70,:,:]
ssv_2014_03 = ssv_anom_ufs_1[70:105,:,:]
ssv_2014_04 = ssv_anom_ufs_1[105:140,:,:]
ssv_2014_05 = ssv_anom_ufs_1[140:175,:,:]
ssv_2014_06 = ssv_anom_ufs_1[175:210,:,:]
ssv_2014_07 = ssv_anom_ufs_1[210:245,:,:]
ssv_2014_08 = ssv_anom_ufs_1[245:280,:,:]
ssv_2014_09 = ssv_anom_ufs_1[280:315,:,:]
ssv_2014_10 = ssv_anom_ufs_1[315:350,:,:]
ssv_2014_11 = ssv_anom_ufs_1[350:385,:,:]
ssv_2014_12 = ssv_anom_ufs_1[385:420,:,:]
ssv_2015_01 = ssv_anom_ufs_1[420:455,:,:]
ssv_2015_02 = ssv_anom_ufs_1[455:490,:,:]
ssv_2015_03 = ssv_anom_ufs_1[490:525,:,:]
ssv_2015_04 = ssv_anom_ufs_1[525:560,:,:]
ssv_2015_05 = ssv_anom_ufs_1[560:595,:,:]
ssv_2015_06 = ssv_anom_ufs_1[595:630,:,:]
ssv_2015_07 = ssv_anom_ufs_1[630:665,:,:]
ssv_2015_08 = ssv_anom_ufs_1[665:700,:,:]
ssv_2015_09 = ssv_anom_ufs_1[700:735,:,:]
ssv_2015_10 = ssv_anom_ufs_1[735:770,:,:]
ssv_2015_11 = ssv_anom_ufs_1[770:805,:,:]
ssv_2015_12 = ssv_anom_ufs_1[805:840,:,:]

In [41]:
taux_2014_01 = taux_anom_ufs_1[0:35,:,:]
taux_2014_02 = taux_anom_ufs_1[35:70,:,:]
taux_2014_03 = taux_anom_ufs_1[70:105,:,:]
taux_2014_04 = taux_anom_ufs_1[105:140,:,:]
taux_2014_05 = taux_anom_ufs_1[140:175,:,:]
taux_2014_06 = taux_anom_ufs_1[175:210,:,:]
taux_2014_07 = taux_anom_ufs_1[210:245,:,:]
taux_2014_08 = taux_anom_ufs_1[245:280,:,:]
taux_2014_09 = taux_anom_ufs_1[280:315,:,:]
taux_2014_10 = taux_anom_ufs_1[315:350,:,:]
taux_2014_11 = taux_anom_ufs_1[350:385,:,:]
taux_2014_12 = taux_anom_ufs_1[385:420,:,:]
taux_2015_01 = taux_anom_ufs_1[420:455,:,:]
taux_2015_02 = taux_anom_ufs_1[455:490,:,:]
taux_2015_03 = taux_anom_ufs_1[490:525,:,:]
taux_2015_04 = taux_anom_ufs_1[525:560,:,:]
taux_2015_05 = taux_anom_ufs_1[560:595,:,:]
taux_2015_06 = taux_anom_ufs_1[595:630,:,:]
taux_2015_07 = taux_anom_ufs_1[630:665,:,:]
taux_2015_08 = taux_anom_ufs_1[665:700,:,:]
taux_2015_09 = taux_anom_ufs_1[700:735,:,:]
taux_2015_10 = taux_anom_ufs_1[735:770,:,:]
taux_2015_11 = taux_anom_ufs_1[770:805,:,:]
taux_2015_12 = taux_anom_ufs_1[805:840,:,:]

In [40]:
tauy_2014_01 = tauy_anom_ufs_1[0:35,:,:]
tauy_2014_02 = tauy_anom_ufs_1[35:70,:,:]
tauy_2014_03 = tauy_anom_ufs_1[70:105,:,:]
tauy_2014_04 = tauy_anom_ufs_1[105:140,:,:]
tauy_2014_05 = tauy_anom_ufs_1[140:175,:,:]
tauy_2014_06 = tauy_anom_ufs_1[175:210,:,:]
tauy_2014_07 = tauy_anom_ufs_1[210:245,:,:]
tauy_2014_08 = tauy_anom_ufs_1[245:280,:,:]
tauy_2014_09 = tauy_anom_ufs_1[280:315,:,:]
tauy_2014_10 = tauy_anom_ufs_1[315:350,:,:]
tauy_2014_11 = tauy_anom_ufs_1[350:385,:,:]
tauy_2014_12 = tauy_anom_ufs_1[385:420,:,:]
tauy_2015_01 = tauy_anom_ufs_1[420:455,:,:]
tauy_2015_02 = tauy_anom_ufs_1[455:490,:,:]
tauy_2015_03 = tauy_anom_ufs_1[490:525,:,:]
tauy_2015_04 = tauy_anom_ufs_1[525:560,:,:]
tauy_2015_05 = tauy_anom_ufs_1[560:595,:,:]
tauy_2015_06 = tauy_anom_ufs_1[595:630,:,:]
tauy_2015_07 = tauy_anom_ufs_1[630:665,:,:]
tauy_2015_08 = tauy_anom_ufs_1[665:700,:,:]
tauy_2015_09 = tauy_anom_ufs_1[700:735,:,:]
tauy_2015_10 = tauy_anom_ufs_1[735:770,:,:]
tauy_2015_11 = tauy_anom_ufs_1[770:805,:,:]
tauy_2015_12 = tauy_anom_ufs_1[805:840,:,:]

Save the months as lists

In [46]:
sst_list = [sst_2014_01, sst_2014_02, sst_2014_03, sst_2014_04, sst_2014_05, sst_2014_06, sst_2014_07,
            sst_2014_08, sst_2014_09, sst_2014_10, sst_2014_11, sst_2014_12, sst_2015_01, sst_2015_02,
            sst_2015_03, sst_2015_04, sst_2015_05, sst_2015_06, sst_2015_07, sst_2015_08, sst_2015_09,
            sst_2015_10, sst_2015_11, sst_2015_12]

In [47]:
sst_list

[<xarray.DataArray 'SST_anom' (time: 35, lat: 721, lon: 1441)>
 dask.array<getitem, shape=(35, 721, 1441), dtype=float32, chunksize=(1, 721, 1441), chunktype=numpy.ndarray>
 Coordinates:
   * lat      (lat) float32 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
   * lon      (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.2 359.5 359.8 360.0
   * time     (time) object 2014-01-01 00:00:00 ... 2014-02-04 00:00:00,
 <xarray.DataArray 'SST_anom' (time: 35, lat: 721, lon: 1441)>
 dask.array<getitem, shape=(35, 721, 1441), dtype=float32, chunksize=(1, 721, 1441), chunktype=numpy.ndarray>
 Coordinates:
   * lat      (lat) float32 -90.0 -89.75 -89.5 -89.25 ... 89.25 89.5 89.75 90.0
   * lon      (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.2 359.5 359.8 360.0
   * time     (time) object 2014-02-01 00:00:00 ... 2014-03-07 00:00:00,
 <xarray.DataArray 'SST_anom' (time: 35, lat: 721, lon: 1441)>
 dask.array<getitem, shape=(35, 721, 1441), dtype=float32, chunksize=(1, 721, 1441), chunktype=numpy.

In [48]:
ssu_list = [ssu_2014_01, ssu_2014_02, ssu_2014_03, ssu_2014_04, ssu_2014_05, ssu_2014_06, ssu_2014_07,
            ssu_2014_08, ssu_2014_09, ssu_2014_10, ssu_2014_11, ssu_2014_12, ssu_2015_01, ssu_2015_02,
            ssu_2015_03, ssu_2015_04, ssu_2015_05, ssu_2015_06, ssu_2015_07, ssu_2015_08, ssu_2015_09,
            ssu_2015_10, ssu_2015_11, ssu_2015_12]

In [49]:
ssv_list = [ssv_2014_01, ssv_2014_02, ssv_2014_03, ssv_2014_04, ssv_2014_05, ssv_2014_06, ssv_2014_07,
            ssv_2014_08, ssv_2014_09, ssv_2014_10, ssv_2014_11, ssv_2014_12, ssv_2015_01, ssv_2015_02,
            ssv_2015_03, ssv_2015_04, ssv_2015_05, ssv_2015_06, ssv_2015_07, ssv_2015_08, ssv_2015_09,
            ssv_2015_10, ssv_2015_11, ssv_2015_12]

In [50]:
taux_list = [taux_2014_01, taux_2014_02, taux_2014_03, taux_2014_04, taux_2014_05, taux_2014_06, taux_2014_07,
            taux_2014_08, taux_2014_09, taux_2014_10, taux_2014_11, taux_2014_12, taux_2015_01, taux_2015_02,
            taux_2015_03, taux_2015_04, taux_2015_05, taux_2015_06, taux_2015_07, taux_2015_08, taux_2015_09,
            taux_2015_10, taux_2015_11, taux_2015_12]

In [51]:
tauy_list = [tauy_2014_01, tauy_2014_02, tauy_2014_03, tauy_2014_04, tauy_2014_05, tauy_2014_06, tauy_2014_07,
            tauy_2014_08, tauy_2014_09, tauy_2014_10, tauy_2014_11, tauy_2014_12, tauy_2015_01, tauy_2015_02,
            tauy_2015_03, tauy_2015_04, tauy_2015_05, tauy_2015_06, tauy_2015_07, tauy_2015_08, tauy_2015_09,
            tauy_2015_10, tauy_2015_11, tauy_2015_12]

In [56]:
len(tauy_list)

24

define the dates for the save files

In [42]:
years = ['2014','2015']

months_num = np.arange(1,13,1)
months = []
for month in months_num: months.append('{:02d}'.format(month))
print(months)

day_init = ['01'] # or ['01']

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']


In [43]:
date_for_file = [f'{year}{month}{day}' for year in years for month in months for day in day_init]
print(date_for_file)

['20140101', '20140201', '20140301', '20140401', '20140501', '20140601', '20140701', '20140801', '20140901', '20141001', '20141101', '20141201', '20150101', '20150201', '20150301', '20150401', '20150501', '20150601', '20150701', '20150801', '20150901', '20151001', '20151101', '20151201']


save function

In [68]:
def save_as(path, dataset, date_range, encoding):
    """
    Given a path, the dataset, the date range, and the encoding: will write out the data as a netcdf and save to the path given.
    
    Make sure to change the variable!
    
    The structure of the function return should have the format:
    return(dataset.to_netcdf((path)+(variable: taux, tauy, SST, SSU, SSV)+"_"+(date_range)+".nc", encoding=encoding))
    """
    
    return(dataset.to_netcdf((path)+"tauy_"+(date_range)+".nc", encoding=encoding))


In [69]:
for i in range(len(date_for_file)):
    save_as(path= "/scratch/ldoyle4/split_anoms/",
            dataset = tauy_list[i],
            date_range = date_for_file[i],
            encoding = {'lat': {'dtype':'float32','_FillValue': None},
                'lon': {'dtype':'float32','_FillValue': None},
                'time': {'dtype':'int32'},
                'tauy_anom':{'dtype':'float32'}
                })